Creates Tensorflow Graphs for spark-nlp DL Annotators and Models


In [ ]:
import numpy as np
import os
import tensorflow as tf
import string
import random
import math
import sys
import shutil

from ner_model import NerModel
from dataset_encoder import DatasetEncoder
from ner_model_saver import NerModelSaver
from pathlib import Path

In [ ]:
def create_graph(ntags, embeddings_dim, nchars, lstm_size = 128):
    tf.reset_default_graph()
    model_name = 'blstm-noncontrib_{}_{}_{}_{}'.format(ntags, embeddings_dim, lstm_size, nchars)
    with tf.Session() as session:
        ner = NerModel(session=None, use_contrib=False)
        ner.add_cnn_char_repr(nchars, 25, 30)
        ner.add_bilstm_char_repr(nchars, 25, 30)
        ner.add_pretrained_word_embeddings(embeddings_dim)
        ner.add_context_repr(ntags, lstm_size, 3)
        ner.add_inference_layer(True)
        ner.add_training_op(5)
        ner.init_variables()
        saver = tf.train.Saver()
        file_name = model_name + '.pb'
        tf.train.write_graph(ner.session.graph, './', file_name, False)
        ner.close()
        session.close()


In [ ]:
create_graph(10, 100, 100)
create_graph(10, 200, 100)
create_graph(10, 300, 100)
create_graph(10, 768, 100)
create_graph(10, 1024, 100)
create_graph(25, 300, 100)